In [1]:
import pandas as pd
import torch
from torch.autograd import Variable
import os
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:

tracknames = os.listdir("/home/jasper-ubuntu/Documents/Studie/Master AI/Computational intelligence/Torcs-CI/CI/train_data/")
# tracknames = ['aalborg.csv']
datadictin = {}
datadictout = {}
framesin =[]
framesout=[]
for i, file in  enumerate(tracknames):
    path = os.path.join('train_data', file)
    print(path)
    datadictin[i] = pd.read_csv(path,sep=';', index_col=False).iloc[:-1, 3:]
    datadictout[i] = pd.read_csv(path,sep=';', index_col=False).iloc[:-1, 0:3]


#Concatenate all datasets
for i,file in enumerate(tracknames):
    framesin.append(datadictin[i])
    framesout.append(datadictout[i])
    
inp = pd.concat(framesin) 
outp = pd.concat(framesout) 

outp.head()

#Drop missing values
inp.replace('', np.nan, inplace=True)
inp.dropna(inplace=True)
outp.replace('', np.nan, inplace=True)
outp.dropna(inplace=True)
outp.head()

train_data/road_alpine-1_7cars_2475231.csv
train_data/road_alpine-1_7cars_2475119.csv
train_data/road_alpine-1_7cars_2475043.csv
train_data/road_alpine-1_7cars_2475155.csv


,accel,brake,steer
0,1.0,0.0,0.0
1,1.0,0.0,0.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,1.0,0.0,0.0


In [39]:
# # Make 1 axis of brake and accelaration
# outp.loc[:,'BRAKE'] *= -1
# outp.iloc[:5,:2]

In [40]:
# maxCol= lambda x: max(x.min(), x.max(), key=abs)
# outp.iloc[:,0:2].apply(maxCol,axis=1)

# outp['brake_acceleration'] = pd.Series(max(abs(outp.iloc[:,0:2])))
outp.head()

,ACCELERATION,BRAKE,STEERING
0,1,0.0,-0.002715
1,1,0.0,-0.002715
2,1,0.0,-0.002715
3,1,0.0,-0.002715
4,1,0.0,-0.002715


In [3]:
colsin = inp.shape[1]
colsout = outp.shape[1]
datapoints = inp.shape[0]
print(datapoints,colsin,colsout)

132708 72 3


In [4]:
# Get data from pandas in a Tensor Variable
x = Variable(torch.from_numpy(inp.iloc[:,:].as_matrix()).float())
y = Variable(torch.from_numpy(outp.iloc[:,:].as_matrix()).float())
print(x)
print(y)

Variable containing:
     0.0000     -0.9820   6306.6499  ...     200.0000    200.0000    200.0000
     0.0000     -0.9620   6306.6499  ...     200.0000    200.0000    200.0000
     0.0000     -0.9420   6306.6499  ...     200.0000    200.0000    200.0000
                ...                   ⋱                   ...                
    -0.0359     90.9260   2000.6600  ...     200.0000    200.0000    200.0000
    -0.0366     90.9480   2000.9600  ...     200.0000    200.0000    200.0000
    -0.0363     90.9700   2001.2700  ...     200.0000    200.0000    200.0000
[torch.FloatTensor of size 132708x72]

Variable containing:
 1.0000  0.0000  0.0000
 1.0000  0.0000  0.0000
 1.0000  0.0000  0.0000
           ⋮            
 1.0000  0.0000 -0.0226
 1.0000  0.0000 -0.0759
 1.0000  0.0000 -0.0557
[torch.FloatTensor of size 132708x3]



In [62]:
class TwoLayerNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        """
        super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, D_out)

    def forward(self, x):
        """
        In the forward function we accept a Variable of input data and we must return
        a Variable of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Variables.
        """
        
        h_relu = self.linear1(x).clamp(min=0)
        y = self.linear2(h_relu)
        y_pred = F.tanh(y)
        return y_pred
    
    
class LSTMNeuralNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        """
        super(LSTMNeuralNet, self).__init__()
        self.hidden_dim = H
        
        self.recurrent = nn.LSTM(D_in, H)
        self.linear = nn.Linear(H, D_out)
        self.hidden = self.init_hidden()
        
    def init_hidden(self):
        """
        Before we've done anything, we dont have any hidden state.
        Refer to the Pytorch documentation to see exactly
        why they have this dimensionality.
        The axes semantics are (num_layers, minibatch_size, hidden_dim)
        """
        return (torch.autograd.Variable(torch.zeros(1, 1, self.hidden_dim)),
                torch.autograd.Variable(torch.zeros(1, 1, self.hidden_dim)))
        
    def forward(self, x):
        """
        In the forward function we accept a Variable of input data and we must return
        a Variable of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Variables.
        """
        lstm_out, self.hidden = self.recurrent(x, self.hidden)
        y = self.linear(lstm_out)
        y_pred = F.tanh(y)
        
        return y_pred


    
# Neural Network Model (1 hidden layer)
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, output_size)
    
    def forward(self, x):
        out = F.sigmoid(self.fc1(x))
        out = F.sigmoid(self.fc2(out))
        out = F.tanh(self.fc3(out))
        return out


In [71]:
class GRUNeuralNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        """
        super(GRUNeuralNet, self).__init__()
        self.hidden_dim = H
        

        self.linear = nn.Linear(H, D_out)
            self.recurrent = nn.GRU(D_in, H, 1)
        self.hidden = self.init_hidden()
        
    def init_hidden(self):
        """
        Before we've done anything, we dont have any hidden state.
        Refer to the Pytorch documentation to see exactly
        why they have this dimensionality.
        The axes semantics are (num_layers, minibatch_size, hidden_dim)
        """
        return (torch.autograd.Variable(torch.zeros(self.hidden_dim, 1, h_layer_size )))
        
    def forward(self, x):
        """
        In the forward function we accept a Variable of input data and we must return
        a Variable of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Variables.
        """
        lstm_out, self.hidden = self.recurrent(x, self.hidden)
        y = self.linear(lstm_out)
        y_pred = F.tanh(y)
        
        return y_pred
    
    

In [63]:

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
# TODO: We doen niets met Batch size..
N, D_in, H, D_out = 50, colsin, 50, colsout

net = RecurrentNeuralNet(D_in, H, D_out)

# mean square error criterion for regression Euclidian distance
criterion = nn.MSELoss()
# criterion.sizeAverage = False

# L1 norm , Manhatan distance
# criterion = nn.L1Loss()

# optimizer = torch.optim.ASGD(net.parameters(), lr=learning_rate)# , momentum=0.9)
lr=0.6
optimizer = torch.optim.SGD([
                {'params': net.parameters(), 'lr': lr}
            ], lr=lr )# , momentum=0.1)

# optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
end=True
test=1
# Train the Model
for epoch in range(15):
   # adjust_learning_rate(optimizer,epoch,lr)
    print("epoch : "+str(epoch))
    if test>0.0001:
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr**epoch
        test=lr**epoch
    elif end:
        end=Flase
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.0001
        test=0.0001
    print("learning rate : "+str(test))
    #print (optimizer.param_group['lr'])
    for i in range(colsin):

        # Clear gradients w.r.t. parameters
        # optimizer.zero_grad()
        # Forward to get output
        out = net(x[i])
        # Calculate Loss
        loss = criterion(out, y[i])
        # print(out[0] - loss[0])
        # print(loss)
        net.zero_grad()
        # Getting gradients w.r.t. parameters
        loss.backward()
        # Updating parameters
        optimizer.step()
#print("iter")

epoch : 0
learning rate : 1.0


RuntimeError: invalid argument 2: dimension 1 out of range of 1D tensor at /opt/conda/conda-bld/pytorch_1503965122592/work/torch/lib/TH/generic/THTensor.c:24

In [72]:
# The nn package also contains definitions of popular loss functions; in this
# case we will use Mean Squared Error (MSE) as our loss function.

learning_rate = 1e-4
net = GRUNeuralNet(D_in, H, D_out)
# The nn package also contains definitions of popular loss functions; in this
# case we will use Mean Squared Error (MSE) as our loss function.
criterion = torch.nn.MSELoss(size_average=False)
optimizer = torch.optim.SGD(net.parameters(), lr=1e-4)
for t in range(15):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = net(x)

    # Compute and print loss
    loss = criterion(y_pred, y)
    print(t, loss.data[0])

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

RuntimeError: invalid argument 2: dimension 1 out of range of 1D tensor at /opt/conda/conda-bld/pytorch_1503965122592/work/torch/lib/TH/generic/THTensor.c:24

In [55]:
# calculate validation set error
#out = net(v_inp)
# Calculate Loss
out = net(x)
loss = criterion(out, y)
print(out[300:600],y[300:600])
# print( out)
# print( v_out)
print("loss : "+str(loss))
# print('Epoch :',epoch, '  Error:', loss)

Variable containing:
   -1     1     1
   -1     1     1
   -1     1     1
   -1     1     1
   -1     1     1
   -1     1     1
   -1     1     1
   -1     1     1
   -1     1     1
   -1     1     1
   -1     1     1
   -1     1     1
   -1     1     1
   -1     1     1
   -1     1     1
   -1     1     1
   -1     1     1
   -1     1     1
   -1     1     1
   -1     1     1
   -1     1     1
   -1     1     1
   -1     1     1
   -1     1     1
   -1     1     1
   -1     1     1
   -1     1     1
   -1     1     1
   -1     1     1
   -1     1     1
   -1     1     1
   -1     1     1
   -1     1     1
   -1     1     1
   -1     1     1
   -1     1     1
   -1     1     1
   -1     1     1
   -1     1     1
   -1     1     1
   -1     1     1
   -1     1     1
   -1     1     1
   -1     1     1
   -1     1     1
   -1     1     1
   -1     1     1
   -1     1     1
   -1     1     1
   -1     1     1
   -1     1     1
   -1     1     1
   -1     1     1
   -1     1     1
   -1  

In [6]:
save_path = 'trainedmodel_4timesalpine1_500hidden_3out(tanh)200it_standardmodel_savedasmodel'
# https://stackoverflow.com/questions/42703500/best-way-to-save-a-trained-model-in-pytorch
torch.save(model, save_path)
# model.save_state_dict('trainedmodel_bbdata_500hidden_3out(tanh)_22in_500it_adam(hopefully_good_results.pt')

In [27]:
neural_net = torch.load('trainedmodel_bbdata_500hidden_3out(tanh)_21in_1000it')

In [59]:
value = x[100]

print(value)
print(net(value))
print(y[100])
# print(loss_fn(model(value),y[100]))

Variable containing:
   -0.0101
    1.0440
 6313.7100
    7.0576
    1.0000
    0.0000
    7.0000
 8935.3701
   50.1658
    0.0429
   -0.0253
    0.3401
    0.2795
   42.2622
   41.8815
   63.4362
   63.4362
    3.9594
    4.9131
    8.2495
   14.0993
   22.9128
   35.5212
   52.9234
   76.0210
  105.2300
  200.0000
  200.0000
  200.0000
  147.3400
   88.0843
   52.6279
   30.7736
   17.4026
   10.1312
    8.0412
  200.0000
  200.0000
  200.0000
  200.0000
  200.0000
  200.0000
  200.0000
  200.0000
  200.0000
  200.0000
  200.0000
  200.0000
  200.0000
  200.0000
  200.0000
  200.0000
  200.0000
   29.8444
   13.4412
   21.8065
    6.9078
  200.0000
  200.0000
  200.0000
  200.0000
  200.0000
  200.0000
  200.0000
  200.0000
  200.0000
  200.0000
  200.0000
  200.0000
  200.0000
  200.0000
  200.0000
[torch.FloatTensor of size 72]

Variable containing:
 0.9859
-0.0046
 0.0030
[torch.FloatTensor of size 3]

Variable containing:
 1.0000
 0.0000
 0.0196
[torch.FloatTensor of size 3]

